### Create environment

In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import gym_kilobots

import pygame
import sys

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [2]:
# Define four objects in a centered square
square_objects = [
    ((-0.25,  0.25), 0.0),   # top-left
    (( 0.25,  0.25), 0.0),   # top-right
    (( 0.25, -0.25), 0.0),   # bottom-right
    ((-0.25, -0.25), 0.0),   # bottom-left
]

# Define 1 object in the center of the environment
single_center_object = [
    ((0.0, 0.0), 0.0)        # center,
]

# Choose a fixed light position
light_pos = (-0.7, 0.1)

# Choose four explicit kilobot positions
kb_positions = [
        (-0.70, 0.10),
        (-0.70, 0.10),
        (-0.75, 0.10),
        (-0.75, 0.10),
    ]

# NOTE: if you don't do this, the environment will randomly place the objects
env = gym.make(
    'Kilobots-QuadAssembly-v0',
    render_mode='human',
    num_kilobots=len(kb_positions),
    object_config=single_center_object,
    light_position=light_pos,
    kilobot_positions=kb_positions
)


In [ ]:
check_env(env,warn=True)

In [ ]:
#vec_env = make_vec_env(env, n_envs = 1)

# Train the agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=300000)
model.save("swarm_ppo")

In [ ]:
model = PPO.load("swarm_ppo")

def controller(x):
      action,states = model.predict(x)
      return action

running = True
obs, info = env.reset()

t = 0


while t <= 100:

      env.render()

      x = obs
      action = controller(x)
      obs, reward, terminated, truncated, info = env.step(action)

      print('Reward: ', reward)

      # Print the orientation of the objects using the unwrapped environment
      #print(env.unwrapped.get_objects_status())

      # End episode
      if terminated or truncated:
            break

      t += 1
      
env.close()  # Ensure the environment is properly closed
pygame.display.quit
pygame.quit()


Reward:  9.328973258579012e-06
Reward:  3.975471442754823e-05
Reward:  0.00015355225297319905
Reward:  0.0005408034064356965
Reward:  0.0017227448246373849
Reward:  0.004992107906993778
Reward:  0.012951495043866078
Reward:  0.03084168447147831
Reward:  0.06571849057884599
Reward:  0.12615644502974305
Reward:  0.21977184081652418
Reward:  0.3713566250689133
Reward:  0.5665955425654441
Reward:  0.7652439274695844
Reward:  0.9222714134150923
Reward:  0.9901293495624147
Reward:  0.9989038384106471
Reward:  0.9996411877909
Reward:  0.9994799806261401
Reward:  0.9998057436286685
Reward:  0.9999769295123552
Reward:  0.9998817937462947
Reward:  0.9996702539361644
Reward:  0.9997631635622215
Reward:  0.9997076392653914
Reward:  0.9995951146668537
Reward:  0.9994679522106874
Reward:  0.9996779363178306
Reward:  0.999513633263522
Reward:  0.9997465159504795
Reward:  0.9999532690543619
Reward:  0.99977016887087
Reward:  0.9999562437990187
Reward:  0.9996079509755657
Reward:  0.9999247160563377
Re

: 